In [4]:
import optuna

In [5]:
from stesml.model_tools import build_train_test_model

In [10]:
def objective(trial):   
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        epochs = 20 # Neglect optmizing # of epochs, allow early stopping to determine # of epochs
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': epochs}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters)
    elif model_type == 'XGBoost':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 50, 10000, log=True)
        parameters = {'n_estimators': n_estimators}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 100, log=True)
        parameters = {'n_estimators': n_estimators}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters)
    
    return result

In [8]:
data_dir = "../data/Sulfur_Models/"
model_type = 'NN' # Options: NN, XGBoost, RandomForest
target = 'Tavg' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'

In [9]:
study = optuna.create_study(direction=direction)

[I 2022-06-30 12:06:53,123] A new study created in memory with name: no-name-97086558-029a-4175-b763-fbe446a807bd


In [6]:
study.optimize(objective, n_trials=50)

2022-06-30 10:45:41.226984: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-30 10:45:41.331334: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/20
12813/12813 [==============================] - 14s 1ms/step - loss: 0.0063 - val_loss: 0.0027
Epoch 2/20
12813/12813 [==============================] - 13s 999us/step - loss: 2.4335e-05 - val_loss: 0.0027
Epoch 3/20
12813/12813 [==============================] - 12s 928us/step - loss: 1.9289e-05 - val_loss: 0.0023
Epoch 4/20
12813/12813 [==============================] - 12s 932us/step - loss: 1.8029e-05 - val_loss: 0.0021
Epoch 5/20
12761/12813 [============================>.] - ETA: 0s - loss: 1.5335e-05

KeyboardInterrupt: 

In [17]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621